In [5]:
# Librairies
import json
import pandas as pd
import numpy as np
import cv2
from collections import defaultdict


# Dossier ou sont sauvegarde les donnee apres le modele
dataFolder = r"C:\Users\faraboli\Desktop\BubbleID\BubbleIDGit\ProjetBubbleID\My_output\SaveData2"
extension = "Test6"
contourFile = dataFolder + "/contours_" + extension +".json"
richFile = dataFolder + "/rich_" + extension +".csv"


def mask_from_contour(contour, shape):
    mask = np.zeros(shape[:2], dtype=np.uint8)
    contour = np.array(contour, dtype=np.int32)
    cv2.fillPoly(mask, [contour], 255)
    return mask

def load_data(json_path, csv_path, shape):
    # Charger JSON
    with open(json_path, 'r') as f:
        json_data = json.load(f)

    # Charger CSV
    df = pd.read_csv(csv_path)
    
    # Associer chaque bulle à son track_id et masque
    data_by_frame = defaultdict(dict)
    for key, contour in json_data.items():
        image_id, bubble_id = map(int, key.split('_'))
        row = df[(df['frame'] == image_id) & (df['det_in_frame'] == bubble_id)]
        if len(row) == 0:
            continue
        track_id = int(row.iloc[0]['track_id'])
        mask = mask_from_contour(contour, shape)
        data_by_frame[image_id][track_id] = mask
    return data_by_frame

def compute_fusions(data_by_frame, threshold=0.01):
    fusions = {}
    sorted_frames = sorted(data_by_frame.keys())
    for i in range(len(sorted_frames) - 1):
        prev_frame = sorted_frames[i]
        next_frame = sorted_frames[i + 1]
        bubbles_prev = data_by_frame[prev_frame]
        bubbles_next = data_by_frame[next_frame]

        for id_next, mask_next in bubbles_next.items():
            intersections = []
            for id_prev, mask_prev in bubbles_prev.items():
                inter = np.logical_and(mask_prev, mask_next)
                inter_area = np.sum(inter)
                prev_area = np.sum(mask_prev)
                if prev_area == 0:
                    continue
                overlap = inter_area / prev_area
                if overlap > threshold:
                    intersections.append(id_prev)
            if len(intersections) == 2:
                fusions[id_next] = intersections
    return fusions


#################################################################
shape = (1024, 1024)  # Taille de l’image
data_by_frame = load_data(contourFile, richFile, shape)
fusions = compute_fusions(data_by_frame)
print("Fusions détectées :")
for new_id, parents in fusions.items():
    print(f"Bulle {new_id} vient de la fusion de {parents}")


Fusions détectées :


In [31]:
import json
import pandas as pd
import numpy as np
import cv2
from collections import defaultdict

# ------------------------
# PARAMÈTRES
# ------------------------
IMAGE_SHAPE = (1024, 1024)
W_PREV = 3
W_NEXT = 3
DILATE_ITERS = 1
KERNEL = np.ones((3, 3), np.uint8)
OVERLAP_THRESH = 0.1


# ------------------------
# UTILITAIRES
# ------------------------
def mask_from_contour(contour, shape):
    mask = np.zeros(shape, dtype=np.uint8)
    if len(contour) == 0:
        return mask
    pts = np.array(contour, dtype=np.int32)
    cv2.fillPoly(mask, [pts], 255)
    if DILATE_ITERS > 0:
        mask = cv2.dilate(mask, KERNEL, iterations=DILATE_ITERS)
    return mask

def overlap_ratio(mask1, mask2):
    inter = np.logical_and(mask1 > 0, mask2 > 0)
    area1 = np.sum(mask1 > 0)
    return np.sum(inter) / area1 if area1 > 0 else 0.0

# ------------------------
# CHARGEMENT DES DONNÉES
# ------------------------
def load_json_contours(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    parsed = {}
    for key, contour in data.items():
        frame_str, det_str = key.split('_')
        frame = int(frame_str)
        det_in_frame = int(det_str)
        parsed[(frame, det_in_frame)] = contour
    return parsed

def build_masks_and_index(json_path, csv_path, image_shape=IMAGE_SHAPE):
    contours = load_json_contours(json_path)
    df = pd.read_csv(csv_path)

    data_by_frame = defaultdict(dict)
    for (frame, det_in_frame), contour in contours.items():
        row = df[(df['frame'] == frame) & (df['det_in_frame'] == det_in_frame)]
        if row.empty:
            continue
        track_id = int(row.iloc[0]['track_id'])
        mask = mask_from_contour(contour, image_shape)
        if np.sum(mask > 0) == 0:
            continue
        data_by_frame[frame][track_id] = mask
    return data_by_frame

# ------------------------
# FONCTION PRINCIPALE
# ------------------------
def detect_fusions(json_path, csv_path, image_shape=IMAGE_SHAPE):
    data_by_frame = build_masks_and_index(json_path, csv_path, image_shape)
    fusion_map = {}  # key = new_track_id, value = {'parents': [...], 'frame': t}

    frames = sorted(data_by_frame.keys())
    for i, frame in enumerate(frames):
        current_bubbles = data_by_frame[frame]
        if i<len(frames)-1:
            next_bubbles = data_by_frame[frame+1]
        prev_frames = [f for f in frames if f < frame-1 and f >= frame - W_PREV]

        previous_bubbles = {}
        for f in prev_frames:
            previous_bubbles.update(data_by_frame[f])

        new_bubbles = [tid for tid in current_bubbles if tid not in previous_bubbles]
        disappeared_bubbles = [tid for tid in previous_bubbles if (tid not in current_bubbles or tid not in next_bubbles)]

        for new_tid in new_bubbles:
            aggregated_mask = np.zeros(image_shape, dtype=np.uint8)
            for f_next in frames:
                if f_next < frame:
                    continue
                if f_next > frame + W_NEXT:
                    break
                if new_tid in data_by_frame[f_next]:
                    aggregated_mask = np.logical_or(aggregated_mask, data_by_frame[f_next][new_tid] > 0)
            aggregated_mask = (aggregated_mask.astype(np.uint8) * 255)

            parents = []
            for parent_tid in disappeared_bubbles:
                parent_mask = previous_bubbles[parent_tid]
                ratio = overlap_ratio(parent_mask, aggregated_mask)
                if ratio > OVERLAP_THRESH:
                    parents.append(parent_tid)

            if parents:
                fusion_map[new_tid] = {'parents': parents, 'frame': frame}

    return fusion_map

# ------------------------
# EXÉCUTION
# ------------------------
# Dossier ou sont sauvegarde les donnee apres le modele
dataFolder = r"C:\Users\faraboli\Desktop\BubbleID\BubbleIDGit\ProjetBubbleID\My_output\SaveData2"
extension = "Test7"
contourFile = dataFolder + "/contours_" + extension +".json"
richFile = dataFolder + "/rich_" + extension +".csv"

fusions = detect_fusions(contourFile, richFile, IMAGE_SHAPE)
print("Fusions détectées :")
for new_bubble, info in fusions.items():
    if len(info['parents'])==2:
        print(f"Frame {info['frame']} : child {new_bubble} ← parents {info['parents']}")



Fusions détectées :
Frame 96 : child 27 ← parents [22, 3]


In [8]:
import json
import pandas as pd
import numpy as np
import cv2
from collections import defaultdict, Counter

# ---------------------------
# Paramètres (à ajuster)
# ---------------------------
IMAGE_SHAPE = (1024, 1024)   # (hauteur, largeur) => adapte selon tes images
WINDOW = 2                   # regarde frames t-W ... t+W
DILATE_ITERS = 2             # dilatation pour compenser contours imparfaits
KERNEL = np.ones((3,3), np.uint8)
OVERLAP_RATIO_THRESH = 0.15  # fraction de l'aire d'une bulle (ou cumulative) pour la considérer candidate
MIN_OVERLAP_PIXELS = 50      # overlap minimal absolu (compense petites bulles)
MAX_PARENTS = 2              # tu as dit max 2 parents
USE_IOU = True              # si True utilise IoU en plus du ratio; sinon ratio relatif à l'aire du parent

# Dossier ou sont sauvegarde les donnee apres le modele
dataFolder = r"C:\Users\faraboli\Desktop\BubbleID\BubbleIDGit\ProjetBubbleID\My_output\SaveData2"
extension = "Test6"
contourFile = dataFolder + "/contours_" + extension +".json"
richFile = dataFolder + "/rich_" + extension +".csv"


# ------------------------
# UTILITAIRES
# ------------------------
def mask_from_contour(contour, shape):
    mask = np.zeros(shape, dtype=np.uint8)
    if len(contour) == 0:
        return mask
    pts = np.array(contour, dtype=np.int32)
    cv2.fillPoly(mask, [pts], 255)
    if DILATE_ITERS > 0:
        mask = cv2.dilate(mask, KERNEL, iterations=DILATE_ITERS)
    return mask

def overlap_relative_to(mask_parent, mask_child):
    a = mask_parent > 0
    c = mask_child > 0
    inter = np.logical_and(a, c).sum()
    parent_area = a.sum()
    return (inter / parent_area if parent_area > 0 else 0.0), inter

def centroid_of_mask(mask):
    ys, xs = np.where(mask > 0)
    if len(xs) == 0:
        return None
    return (int(np.mean(xs)), int(np.mean(ys)))  # (x, y)

# ------------------------
# CHARGEMENT
# ------------------------
def load_json_contours(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    parsed = {}
    for key, contour in data.items():
        frame_str, det_str = key.split('_')
        frame = int(frame_str)
        det_in_frame = int(det_str)
        parsed[(frame, det_in_frame)] = contour
    return parsed

def build_masks_and_index(json_path, csv_path, image_shape=IMAGE_SHAPE):
    contours = load_json_contours(json_path)
    df = pd.read_csv(csv_path).rename(
        columns={"numero_image": "frame", "numero_bulle": "det_in_frame"}
    )

    data_by_frame = defaultdict(dict)
    for (frame, det_in_frame), contour in contours.items():
        row = df[(df['frame'] == frame) & (df['det_in_frame'] == det_in_frame)]
        if row.empty:
            continue
        track_id = int(row.iloc[0]['track_id'])
        mask = mask_from_contour(contour, image_shape)
        data_by_frame[frame][track_id] = {
            'mask': mask,
            'area': np.sum(mask > 0),
            'centroid': centroid_of_mask(mask)
        }
    return data_by_frame

# ------------------------
# DÉTECTION DE FUSIONS
# ------------------------
def detect_fusions(data_by_frame, window=WINDOW):
    fusions = {}
    frames = sorted(data_by_frame.keys())

    for i, frame in enumerate(frames):
        # Ignorer la première frame (pas de parents possibles)
        if i == 0:
            continue

        frame_data = data_by_frame[frame]
        # Frames précédentes dans la fenêtre temporelle
        prev_frames = [f for f in frames if f < frame and f >= frame - window]

        for child_tid, child_info in frame_data.items():
            mask_child = child_info['mask']
            total_overlap = Counter()

            # Recherche de parents dans les frames précédentes
            for pf in prev_frames:
                for parent_tid, parent_info in data_by_frame[pf].items():
                    # Exclure soi-même
                    if parent_tid == child_tid:
                        continue

                    mask_parent = parent_info['mask']
                    ratio, inter_pix = overlap_relative_to(mask_parent, mask_child)

                    if ratio > OVERLAP_RATIO_THRESH or inter_pix > MIN_OVERLAP_PIXELS:
                        total_overlap[parent_tid] += ratio

            # Garder les meilleurs parents
            if len(total_overlap) > 0:
                top_parents = [
                    tid for tid, _ in total_overlap.most_common(MAX_PARENTS)
                ]
                # Seulement si on a ≥2 parents => vraie fusion
                if len(top_parents) == 2:
                    fusions[(frame, child_tid)] = top_parents

    return fusions

# ------------------------
# EXÉCUTION
# ------------------------


data_by_frame = build_masks_and_index(contourFile, richFile, IMAGE_SHAPE)
fusions = detect_fusions(data_by_frame, window=2)

print("Fusions détectées :")
for (frame, child), parents in fusions.items():
    print(f"Frame {frame} : enfant {child} ← parents {parents}")


Fusions détectées :
Frame 2 : enfant 10 ← parents [12, 13]
Frame 2 : enfant 12 ← parents [10, 13]
Frame 2 : enfant 13 ← parents [11, 4]
Frame 3 : enfant 10 ← parents [12, 13]
Frame 3 : enfant 11 ← parents [13, 4]
Frame 3 : enfant 12 ← parents [10, 13]
Frame 4 : enfant 1 ← parents [14, 16]
Frame 4 : enfant 16 ← parents [5, 17]
Frame 4 : enfant 17 ← parents [5, 16]
Frame 4 : enfant 10 ← parents [12, 13]
Frame 4 : enfant 11 ← parents [13, 4]
Frame 4 : enfant 18 ← parents [13, 11]
Frame 4 : enfant 12 ← parents [10, 13]
Frame 5 : enfant 1 ← parents [16, 5]
Frame 5 : enfant 16 ← parents [5, 17]
Frame 5 : enfant 8 ← parents [18, 11]
Frame 5 : enfant 10 ← parents [12, 11]
Frame 5 : enfant 11 ← parents [4, 12]
Frame 5 : enfant 12 ← parents [10, 11]
Frame 6 : enfant 8 ← parents [18, 19]
Frame 6 : enfant 10 ← parents [12, 11]
Frame 6 : enfant 18 ← parents [19, 11]
Frame 6 : enfant 12 ← parents [10, 11]
Frame 7 : enfant 10 ← parents [12, 11]
Frame 7 : enfant 8 ← parents [18, 19]
Frame 7 : enfant 1